In [5]:
import numpy as np
import pandas as pd
import xarray as xr
import zipfile
import glob
import netCDF4 as nc
import xesmf as xe
from scipy import stats

def trend_finder(timeseries_maps):
    """
    Find decadal trends at each lat and lon point. 
    """
    shape_of_maps = np.shape(timeseries_maps)
    # must be adjusted based on len of time used for trends
    years = np.linspace(0,3.5,36)
    trend_map = []
    for lat in range(0, shape_of_maps[1]):
        lat_line = []
        for lon in range(0, shape_of_maps[2]):
            timeseries = timeseries_maps[:,lat,lon]
            trend = stats.linregress(x=years, y=timeseries)[0] # in K/dec
            lat_line.append(trend)
        trend_map.append(lat_line)
    return(np.array(trend_map))
    
            
# load path to CMIP 6 data
path_to_CMIP6_data = '/home/disk/pna2/aodhan/CMIP6/historical_monthly_tas_google/*'
model_paths = glob.glob(path_to_CMIP6_data)

# here we select the start times of trends, this can be changed
start_times = np.arange(1854,1980,5)

# for each model...
for model in model_paths:
    realizations = glob.glob(model + '/*')

    # create data dictionary
    model_trends = []
    
    # for each realization...
    simulation_names = []
    for ensemble_member in realizations:
        simulation = xr.open_dataset(ensemble_member)
        model_name = simulation.source_id
        simulation_name = simulation.variant_label
        simulation_names.append(simulation_name)
        print(model_name, simulation_name)

         # CMIP6 models must be regridded, below we define input and output grids
        latitudes = simulation.lat.values
        longitudes = simulation.lon.values
        InputGrid = {"lon": longitudes, "lat": latitudes}
        OutputGrid = {"lon": np.arange(1.25, 358.751, 2.5), "lat": np.arange(-88.75, 88.751, 2.5)}
        regridder = xe.Regridder(InputGrid, OutputGrid, "bilinear")
        
        # create lists for where to append data
        all_timeperiod_trends = []
        time_keys = []

        # find time slices for just winter
        for i in start_times:
            start_date = str(i) + '-01'
            end_date = str(i+35) + '-12'
            time_key = str(i) + '-' + str(i+35)

            # find correct time period of data
            time_slice_data = simulation.sel(time=slice(start_date, end_date))
            data_array = time_slice_data.tas.values

            # find trends for each of the time periods
            shape_of_data_array = np.shape(data_array)
            data_calendar = np.reshape(data_array, (36, 12, shape_of_data_array[1], shape_of_data_array[2]))
            season_calendar = [data_calendar[:,10],data_calendar[:,11], data_calendar[:,0], 
                               data_calendar[:,1], data_calendar[:,2]]
            timeseries_map = np.nanmean(season_calendar, axis=0)
            trend_map = trend_finder(timeseries_map)
            trend_map_2p5x2p5 = regridder(trend_map)
            all_timeperiod_trends.append(trend_map_2p5x2p5)
            time_keys.append(time_key)
        
        # append all timeperiod trends 
        model_trends.append(all_timeperiod_trends)

    

    # Timeperiod data will be dumped into NetCDF files
    fileName = path_to_CMIP6_data + '/trendmap' + model.replace("-", "_") + '_TAS_NDJFM_TrendMaps.nc'

    # Create netcdf file with dimensions
    ds = nc.Dataset(fileName, 'w', format='NETCDF4')
    ensemble_member = ds.createDimension('ensemble_member', len(simulation_names))
    TrendTimePeriod = ds.createDimension('TrendTimePeriod', 26) # 26 timeperiods
    Lat = ds.createDimension('Lat', 72)
    Lon = ds.createDimension('Lon', 144)

    # Add variables to dimensions
    ensemble_member = ds.createVariable('ensemble_member', str, ('ensemble_member',))
    TrendTimePeriod = ds.createVariable('TrendTimePeriod', str, ('TrendTimePeriod',))
    Lat = ds.createVariable('Lat', 'f4', ('Lat',))
    Lon = ds.createVariable('Lon', 'f4', ('Lon',))
    Ts_trends = ds.createVariable('ts_trend', 'f4', ('ensemble_member', 'TrendTimePeriod', 'Lat', 'Lon'))

    # Assing values to variables
    ensemble_member[:] = simulation_names
    TrendTimePeriod[:] = time_keys
    Lat[:] = np.arange(-88.75, 88.751, 2.5)
    Lon[:] = np.arange(1.25, 358.751, 2.5)
    Ts_trends[:] = model_trends

    ds.close()
    break

IPSL-CM6A-LR r8i1p1f1
IPSL-CM6A-LR r2i1p1f1
IPSL-CM6A-LR r30i1p1f1
IPSL-CM6A-LR r29i1p1f1
IPSL-CM6A-LR r3i1p1f1
IPSL-CM6A-LR r6i1p1f1
IPSL-CM6A-LR r27i1p1f1
IPSL-CM6A-LR r7i1p1f1
IPSL-CM6A-LR r9i1p1f1
IPSL-CM6A-LR r26i1p1f1
IPSL-CM6A-LR r20i1p1f1
IPSL-CM6A-LR r25i1p1f1
IPSL-CM6A-LR r24i1p1f1
IPSL-CM6A-LR r31i1p1f1
IPSL-CM6A-LR r22i1p1f1
IPSL-CM6A-LR r21i1p1f1
IPSL-CM6A-LR r23i1p1f1
IPSL-CM6A-LR r19i1p1f1
IPSL-CM6A-LR r18i1p1f1
IPSL-CM6A-LR r11i1p1f1
IPSL-CM6A-LR r12i1p1f1
IPSL-CM6A-LR r17i1p1f1
IPSL-CM6A-LR r16i1p1f1
IPSL-CM6A-LR r1i1p1f1
IPSL-CM6A-LR r5i1p1f1
IPSL-CM6A-LR r4i1p1f1
IPSL-CM6A-LR r28i1p1f1
IPSL-CM6A-LR r14i1p1f1
IPSL-CM6A-LR r10i1p1f1
IPSL-CM6A-LR r15i1p1f1


In [4]:
len(start_times)

26